## Multioutput regression 

- Captures correlations between related targets , the model learns joint patterns
- More efficient, train once instead of multiple times
- Can improve performance, related outputs help regularize each other
- Shares feature importance across targets

In [58]:
import numpy as np
import pandas as pd
import librosa
from pathlib import Path
from tqdm import tqdm
import pickle
import hashlib

In [59]:
# Configuration - SET THIS ACCORDING TO YOUR ENVIRONMENT
RUNNING_ON_WSL = False  

if RUNNING_ON_WSL:
    # Running on Linux/WSL
    PROJECT_ROOT = Path("/home/rime/music-recom")
else:
    # Running on Windows accessing WSL files
    PROJECT_ROOT = Path(r"\\wsl.localhost\Ubuntu-22.04\home\rime\music-recom")

DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"
CACHE_DIR = PROCESSED_DIR / "cache"


In [60]:
def fix_audio_path(path_str, running_on_wsl=RUNNING_ON_WSL):
    """Convert stored path to the correct format for current environment"""
    # Extract just the filename
    filename = Path(path_str).name
    
    # Build correct path
    correct_path = DATA_DIR / "processed" / 'audio' / filename
    
    return correct_path

Cache files

In [61]:
# Create directories if they don't exist
CACHE_DIR.mkdir(parents=True, exist_ok=True)

def get_cache_key(audio_path, n_mfcc, sr):
    """Generate unique cache key for audio file"""
    file_stat = Path(audio_path).stat()
    cache_string = f"{audio_path}_{n_mfcc}_{sr}_{file_stat.st_size}_{file_stat.st_mtime}"
    return hashlib.md5(cache_string.encode()).hexdigest()

def load_from_cache(cache_key):
    """Load features from cache if available"""
    cache_file = CACHE_DIR / f"{cache_key}.pkl"
    if cache_file.exists():
        with open(cache_file, 'rb') as f:
            return pickle.load(f)
    return None

def save_to_cache(cache_key, features):
    """Save features to cache"""
    cache_file = CACHE_DIR / f"{cache_key}.pkl"
    with open(cache_file, 'wb') as f:
        pickle.dump(features, f)

In [62]:
def extract_mfcc_features(audio_path, n_mfcc=13, sr=22050, use_cache=True):
    try:
        # Check cache first
        if use_cache:
            cache_key = get_cache_key(audio_path, n_mfcc, sr)
            cached_features = load_from_cache(cache_key)
            if cached_features is not None:
                return cached_features, True
        
        # Load audio
        signal, sr = librosa.load(audio_path, sr=sr)
        
        # Extract MFCCs
        mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc)
        
        # Extract deltas
        delta_mfccs = librosa.feature.delta(mfccs)
        delta2_mfccs = librosa.feature.delta(mfccs, order=2)
        
        # Compute statistics
        features = {}
        
        # MFCC statistics
        features['mfcc_mean'] = np.mean(mfccs, axis=1)
        features['mfcc_std'] = np.std(mfccs, axis=1)
        features['mfcc_q25'] = np.percentile(mfccs, 25, axis=1)
        features['mfcc_q50'] = np.percentile(mfccs, 50, axis=1)  # median
        features['mfcc_q75'] = np.percentile(mfccs, 75, axis=1)
        
        # Delta statistics
        features['delta_mean'] = np.mean(delta_mfccs, axis=1)
        features['delta_std'] = np.std(delta_mfccs, axis=1)
        features['delta_q25'] = np.percentile(delta_mfccs, 25, axis=1)
        features['delta_q50'] = np.percentile(delta_mfccs, 50, axis=1)
        features['delta_q75'] = np.percentile(delta_mfccs, 75, axis=1)
        
        # Delta2 statistics
        features['delta2_mean'] = np.mean(delta2_mfccs, axis=1)
        features['delta2_std'] = np.std(delta2_mfccs, axis=1)
        features['delta2_q25'] = np.percentile(delta2_mfccs, 25, axis=1)
        features['delta2_q50'] = np.percentile(delta2_mfccs, 50, axis=1)
        features['delta2_q75'] = np.percentile(delta2_mfccs, 75, axis=1)
        
        # Flatten all features into a single vector
        feature_vector = np.concatenate([
            features['mfcc_mean'], features['mfcc_std'],
            features['mfcc_q25'], features['mfcc_q50'], features['mfcc_q75'],
            features['delta_mean'], features['delta_std'],
            features['delta_q25'], features['delta_q50'], features['delta_q75'],
            features['delta2_mean'], features['delta2_std'],
            features['delta2_q25'], features['delta2_q50'], features['delta2_q75']
        ])
        
        # Save to cache
        if use_cache:
            save_to_cache(cache_key, feature_vector)
        
        return feature_vector, True
        
    except Exception as e:
        print(f"Error processing {audio_path}: {str(e)}")
        return None, False

In [63]:
def save_checkpoint(ids, vectors, n_mfcc, checkpoint_path, existing_df=None):
    """Save progress checkpoint"""
    stats = ['mean', 'std', 'q25', 'q50', 'q75']
    feature_names = []
    for prefix in ['mfcc', 'delta', 'delta2']:
        for stat in stats:
            for i in range(n_mfcc):
                feature_names.append(f"{prefix}_{stat}_{i}")
    
    new_df = pd.DataFrame(vectors, columns=feature_names)
    new_df.insert(0, 'track_id', ids)
    
    if existing_df is not None:
        checkpoint_df = pd.concat([existing_df, new_df], ignore_index=True)
    else:
        checkpoint_df = new_df
    
    checkpoint_df.to_csv(checkpoint_path, index=False)
    print(f"\nCheckpoint saved: {len(checkpoint_df)} songs processed")

In [64]:
def extract_features_for_dataset(df, n_mfcc=13, sr=22050, use_cache=True, 
                                checkpoint_interval=100, checkpoint_path=None):
    if checkpoint_path is None:
        checkpoint_path = PROCESSED_DIR / "mfcc_checkpoint.csv"
    
    # Try to load existing checkpoint
    if checkpoint_path.exists():
        print(f"Found checkpoint at {checkpoint_path}")
        existing_df = pd.read_csv(checkpoint_path)
        processed_ids = set(existing_df['track_id'].values)
        print(f"Resuming from checkpoint with {len(processed_ids)} already processed songs")
    else:
        existing_df = None
        processed_ids = set()
    
    successful_ids = []
    feature_vectors = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting MFCC features"):
        track_id = row['track_id']
        audio_path_from_csv = row['audio_path']
        
        # Skip if already processed
        if track_id in processed_ids:
            continue
        
        # Convert to correct format for current environment
        audio_path = fix_audio_path(audio_path_from_csv)
        
        # Extract features
        features, success = extract_mfcc_features(audio_path, n_mfcc=n_mfcc, 
                                                  sr=sr, use_cache=use_cache)
        
        if success:
            successful_ids.append(track_id)
            feature_vectors.append(features)
            
            # Save checkpoint periodically
            if len(successful_ids) % checkpoint_interval == 0:
                save_checkpoint(successful_ids, feature_vectors, n_mfcc, 
                              checkpoint_path, existing_df)
    
    # Generate column names
    feature_names = []
    stats = ['mean', 'std', 'q25', 'q50', 'q75']
    for prefix in ['mfcc', 'delta', 'delta2']:
        for stat in stats:
            for i in range(n_mfcc):
                feature_names.append(f"{prefix}_{stat}_{i}")
    
    # Create new features DataFrame
    new_features_df = pd.DataFrame(feature_vectors, columns=feature_names)
    new_features_df.insert(0, 'track_id', successful_ids)
    
    # Combine with existing if checkpoint exists
    if existing_df is not None:
        features_df = pd.concat([existing_df, new_features_df], ignore_index=True)
    else:
        features_df = new_features_df
    
    print(f"\n✓ Successfully extracted features for {len(features_df)}/{len(df)} songs")
    print(f"✓ Feature vector dimension: {len(feature_names)} ({n_mfcc} MFCCs × 5 statistics × 3 types)")
    
    # Remove checkpoint file after successful completion
    if checkpoint_path.exists():
        checkpoint_path.unlink()
        print(f"✓ Removed checkpoint file")
    
    return features_df

In [65]:
# Main execution
if __name__ == "__main__":
    # Load audio files list
    audio_list_path = PROCESSED_DIR / "audio_files_list.csv"
    
    print(f"Loading data from: {audio_list_path}")
    print(f"Environment: {'WSL/Linux' if RUNNING_ON_WSL else 'Windows accessing WSL'}")
    
    id_music_list_df = pd.read_csv(audio_list_path)
    
    print(f"\nLoaded {len(id_music_list_df)} audio files")
    print(f"Sample path: {id_music_list_df['audio_path'].iloc[0]}")
    
    # Test path conversion
    test_path = fix_audio_path(id_music_list_df['audio_path'].iloc[0])
    print(f"Converted to: {test_path}")
    print(f"Path exists: {test_path.exists()}")
    
    if not test_path.exists():
        print("\nWARNING: Test path doesn't exist!")
        print("Please check:")
        print("  1. Is RUNNING_ON_WSL set correctly?")
        print("  2. Are the files accessible at the WSL location?")
        response = input("\nContinue anyway? (y/n): ")
        if response.lower() != 'y':
            print("Exiting...")
            exit()
    
    # Extract features
    mfcc_features_df = extract_features_for_dataset(
        id_music_list_df, 
        n_mfcc=13, 
        sr=22050,
        use_cache=True,
        checkpoint_interval=100
    )
    
    # Save results
    output_path = PROCESSED_DIR / "mfcc_features_with_quartiles.csv"
    mfcc_features_df.to_csv(output_path, index=False)
    print(f"\n✓ Saved features to {output_path}")
    
    print(f"\nFeature extraction summary:")
    print(f"  Total features per song: {len(mfcc_features_df.columns) - 1}")
    print(f"  Successfully processed: {len(mfcc_features_df)} songs")
    print(f"  Cache directory: {CACHE_DIR}")

Loading data from: \\wsl.localhost\Ubuntu-22.04\home\rime\music-recom\data\processed\audio_files_list.csv
Environment: Windows accessing WSL

Loaded 1590 audio files
Sample path: \home\rime\music-recom\data\audio\109497.mp3
Converted to: \\wsl.localhost\Ubuntu-22.04\home\rime\music-recom\data\processed\audio\109497.mp3
Path exists: True


Extracting MFCC features:   7%|▋         | 110/1590 [00:01<00:28, 52.15it/s]


Checkpoint saved: 100 songs processed


Extracting MFCC features:  13%|█▎        | 206/1590 [00:03<00:28, 48.75it/s]


Checkpoint saved: 200 songs processed


...


Checkpoint saved: 1400 songs processed


Extracting MFCC features:  95%|█████████▍| 1510/1590 [00:29<00:02, 31.94it/s]


Checkpoint saved: 1500 songs processed


Extracting MFCC features: 100%|██████████| 1590/1590 [00:31<00:00, 50.70it/s]



✓ Successfully extracted features for 1590/1590 songs
✓ Feature vector dimension: 195 (13 MFCCs × 5 statistics × 3 types)
✓ Removed checkpoint file

✓ Saved features to \\wsl.localhost\Ubuntu-22.04\home\rime\music-recom\data\processed\mfcc_features_with_quartiles.csv

Feature extraction summary:
  Total features per song: 195
  Successfully processed: 1590 songs
  Cache directory: \\wsl.localhost\Ubuntu-22.04\home\rime\music-recom\data\processed\cache


### Grouping similar features and predicting them via a Random Forest multioutput regressor 

In [66]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

In [67]:
# Rythm/ Movement targets
rythm_features = ['tempo', 'danceability', 'energy']

# Audio characteristics
audio_features = ['acousticness', 'liveness', 'loudness']

# Vocal/ content features
vocal_features = ['speechiness', 'valence', 'intrementalness']

In [68]:
# Get the current working directory 
root = Path.cwd().parent

In [69]:
pd.set_option('display.max_columns', None)

In [70]:
# Build the full path
MFCC_Q_df = pd.read_csv(root / 'data/processed/mfcc_features_with_quartiles.csv')

MFCC_Q_df.head()

,track_id,mfcc_mean_0,mfcc_mean_1,mfcc_mean_2,mfcc_mean_3,mfcc_mean_4,mfcc_mean_5,mfcc_mean_6,mfcc_mean_7,mfcc_mean_8,mfcc_mean_9,mfcc_mean_10,mfcc_mean_11,mfcc_mean_12,mfcc_std_0,mfcc_std_1,mfcc_std_2,mfcc_std_3,mfcc_std_4,mfcc_std_5,mfcc_std_6,mfcc_std_7,mfcc_std_8,mfcc_std_9,mfcc_std_10,mfcc_std_11,mfcc_std_12,mfcc_q25_0,mfcc_q25_1,mfcc_q25_2,mfcc_q25_3,mfcc_q25_4,mfcc_q25_5,mfcc_q25_6,mfcc_q25_7,mfcc_q25_8,mfcc_q25_9,mfcc_q25_10,mfcc_q25_11,mfcc_q25_12,mfcc_q50_0,mfcc_q50_1,mfcc_q50_2,mfcc_q50_3,mfcc_q50_4,mfcc_q50_5,mfcc_q50_6,mfcc_q50_7,mfcc_q50_8,mfcc_q50_9,mfcc_q50_10,mfcc_q50_11,mfcc_q50_12,mfcc_q75_0,mfcc_q75_1,mfcc_q75_2,mfcc_q75_3,mfcc_q75_4,mfcc_q75_5,mfcc_q75_6,mfcc_q75_7,mfcc_q75_8,mfcc_q75_9,mfcc_q75_10,mfcc_q75_11,mfcc_q75_12,delta_mean_0,delta_mean_1,delta_mean_2,delta_mean_3,delta_mean_4,delta_mean_5,delta_mean_6,delta_mean_7,delta_mean_8,delta_mean_9,delta_mean_10,delta_mean_11,delta_mean_12,delta_std_0,delta_std_1,delta_std_2,delta_std_3,delta_std_4,delta_std_5,delta_std_6,delta_std_7,delta_std_8,delta_std_9,delta_std_10,delta_std_11,delta_std_12,delta_q25_0,delta_q25_1,delta_q25_2,delta_q25_3,delta_q25_4,delta_q25_5,delta_q25_6,delta_q25_7,delta_q25_8,delta_q25_9,delta_q25_10,delta_q25_11,delta_q25_12,delta_q50_0,delta_q50_1,delta_q50_2,delta_q50_3,delta_q50_4,delta_q50_5,delta_q50_6,delta_q50_7,delta_q50_8,delta_q50_9,delta_q50_10,delta_q50_11,delta_q50_12,delta_q75_0,delta_q75_1,delta_q75_2,delta_q75_3,delta_q75_4,delta_q75_5,delta_q75_6,delta_q75_7,delta_q75_8,delta_q75_9,delta_q75_10,delta_q75_11,delta_q75_12,delta2_mean_0,delta2_mean_1,delta2_mean_2,delta2_mean_3,delta2_mean_4,delta2_mean_5,delta2_mean_6,delta2_mean_7,delta2_mean_8,delta2_mean_9,delta2_mean_10,delta2_mean_11,delta2_mean_12,delta2_std_0,delta2_std_1,delta2_std_2,delta2_std_3,delta2_std_4,delta2_std_5,delta2_std_6,delta2_std_7,delta2_std_8,delta2_std_9,delta2_std_10,delta2_std_11,delta2_std_12,delta2_q25_0,delta2_q25_1,delta2_q25_2,delta2_q25_3,delta2_q25_4,delta2_q25_5,delta2_q25_6,delta2_q25_7,delta2_q25_8,delta2_q25_9,delta2_q25_10,delta2_q25_11,delta2_q25_12,delta2_q50_0,delta2_q50_1,delta2_q50_2,delta2_q50_3,delta2_q50_4,delta2_q50_5,delta2_q50_6,delta2_q50_7,delta2_q50_8,delta2_q50_9,delta2_q50_10,delta2_q50_11,delta2_q50_12,delta2_q75_0,delta2_q75_1,delta2_q75_2,delta2_q75_3,delta2_q75_4,delta2_q75_5,delta2_q75_6,delta2_q75_7,delta2_q75_8,delta2_q75_9,delta2_q75_10,delta2_q75_11,delta2_q75_12
0,109497,-14.466280,99.482414,-13.127948,43.313545,9.995256,5.399690,-5.793285,13.883069,-1.838129,2.187864,-2.674747,3.841628,-9.301169,26.336906,13.115074,12.082673,10.713166,7.633686,5.559512,5.984026,5.907621,5.963255,5.159916,5.725162,4.790164,5.414805,-25.974308,89.797730,-21.092014,38.467827,6.111401,1.586030,-9.500146,9.957172,-5.797652,-1.314428,-6.898641,0.665738,-12.874005,-12.028065,98.771830,-13.556653,44.643456,11.083105,5.215743,-5.622651,14.334005,-1.961504,2.256998,-2.872006,3.641615,-9.346977,0.913519,107.980170,-5.873160,50.600290,15.318432,9.029509,-1.505398,18.285854,1.977354,5.757357,1.137958,7.034313,-5.877173,0.220803,0.009053,-0.002882,-0.003425,0.000750,0.004600,-0.003279,0.011742,0.006375,0.001650,0.000165,-0.000724,0.005746,4.416480,2.527710,1.879202,1.614143,1.372204,0.916860,1.072609,1.153425,1.141011,0.906527,1.142041,0.910949,1.012290,-2.169072,-1.664055,-1.188237,-1.053944,-0.914239,-0.635814,-0.725844,-0.817876,-0.767006,-0.613866,-0.804201,-0.617282,-0.637065,0.089583,0.031641,0.065600,0.032003,0.038310,-0.001774,-0.009710,0.056556,0.040189,-0.014482,-0.020248,0.000350,0.008368,2.173461,1.705773,1.284936,1.063335,0.875187,0.629731,0.694629,0.863577,0.787185,0.644893,0.815830,0.614444,0.648924,-0.135776,-0.035974,0.004075,-0.013851,-0.004475,-0.001072,-0.002500,-0.009171,-0.006136,-0.009151,-0.006148,-0.006779,-0.001375,2.662576,1.463119,1.183564,1.022441,0.859411,0.726815,0.796010,0.860587,0.880833,0.730460,0.814752,0.704768,0.741921,-1.265242,-0.991705,-0.796757,-0.670792,-0.626229,-0.487701,-0.562620,-0.

In [71]:
matched_df = pd.read_csv(root / 'data/processed/matched_metadata.csv')

continuous_targets = [ 'danceability', 'energy',  
           'loudness', 'speechiness', 'acousticness', 
           'instrumentalness', 'liveness', 'valence', 'tempo']

matched_df_target = matched_df[['track_id'] + continuous_targets]
matched_df_target.head()

,track_id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,10,0.606,0.916,-8.162,0.0371,0.1400,0.356,0.1320,0.8890,111.563
1,237,0.280,0.640,-7.799,0.1230,0.3490,0.675,0.1360,0.0537,140.368
2,238,0.192,0.411,-9.445,0.0655,0.5390,0.709,0.0909,0.1390,56.929
3,459,0.584,0.918,-9.883,0.0345,0.0254,0.770,0.3480,0.1140,108.305
4,459,0.415,0.646,-12.022,0.0399,0.0189,0.948,0.0965,0.1230,93.887


In [72]:
c_targets_df = matched_df_target[["track_id"]+ continuous_targets]
merged_df = MFCC_Q_df.merge(c_targets_df, on = 'track_id')

merged_df.head()

,track_id,mfcc_mean_0,mfcc_mean_1,mfcc_mean_2,mfcc_mean_3,mfcc_mean_4,mfcc_mean_5,mfcc_mean_6,mfcc_mean_7,mfcc_mean_8,mfcc_mean_9,mfcc_mean_10,mfcc_mean_11,mfcc_mean_12,mfcc_std_0,mfcc_std_1,mfcc_std_2,mfcc_std_3,mfcc_std_4,mfcc_std_5,mfcc_std_6,mfcc_std_7,mfcc_std_8,mfcc_std_9,mfcc_std_10,mfcc_std_11,mfcc_std_12,mfcc_q25_0,mfcc_q25_1,mfcc_q25_2,mfcc_q25_3,mfcc_q25_4,mfcc_q25_5,mfcc_q25_6,mfcc_q25_7,mfcc_q25_8,mfcc_q25_9,mfcc_q25_10,mfcc_q25_11,mfcc_q25_12,mfcc_q50_0,mfcc_q50_1,mfcc_q50_2,mfcc_q50_3,mfcc_q50_4,mfcc_q50_5,mfcc_q50_6,mfcc_q50_7,mfcc_q50_8,mfcc_q50_9,mfcc_q50_10,mfcc_q50_11,mfcc_q50_12,mfcc_q75_0,mfcc_q75_1,mfcc_q75_2,mfcc_q75_3,mfcc_q75_4,mfcc_q75_5,mfcc_q75_6,mfcc_q75_7,mfcc_q75_8,mfcc_q75_9,mfcc_q75_10,mfcc_q75_11,mfcc_q75_12,delta_mean_0,delta_mean_1,delta_mean_2,delta_mean_3,delta_mean_4,delta_mean_5,delta_mean_6,delta_mean_7,delta_mean_8,delta_mean_9,delta_mean_10,delta_mean_11,delta_mean_12,delta_std_0,delta_std_1,delta_std_2,delta_std_3,delta_std_4,delta_std_5,delta_std_6,delta_std_7,delta_std_8,delta_std_9,delta_std_10,delta_std_11,delta_std_12,delta_q25_0,delta_q25_1,delta_q25_2,delta_q25_3,delta_q25_4,delta_q25_5,delta_q25_6,delta_q25_7,delta_q25_8,delta_q25_9,delta_q25_10,delta_q25_11,delta_q25_12,delta_q50_0,delta_q50_1,delta_q50_2,delta_q50_3,delta_q50_4,delta_q50_5,delta_q50_6,delta_q50_7,delta_q50_8,delta_q50_9,delta_q50_10,delta_q50_11,delta_q50_12,delta_q75_0,delta_q75_1,delta_q75_2,delta_q75_3,delta_q75_4,delta_q75_5,delta_q75_6,delta_q75_7,delta_q75_8,delta_q75_9,delta_q75_10,delta_q75_11,delta_q75_12,delta2_mean_0,delta2_mean_1,delta2_mean_2,delta2_mean_3,delta2_mean_4,delta2_mean_5,delta2_mean_6,delta2_mean_7,delta2_mean_8,delta2_mean_9,delta2_mean_10,delta2_mean_11,delta2_mean_12,delta2_std_0,delta2_std_1,delta2_std_2,delta2_std_3,delta2_std_4,delta2_std_5,delta2_std_6,delta2_std_7,delta2_std_8,delta2_std_9,delta2_std_10,delta2_std_11,delta2_std_12,delta2_q25_0,delta2_q25_1,delta2_q25_2,delta2_q25_3,delta2_q25_4,delta2_q25_5,delta2_q25_6,delta2_q25_7,delta2_q25_8,delta2_q25_9,delta2_q25_10,delta2_q25_11,delta2_q25_12,delta2_q50_0,delta2_q50_1,delta2_q50_2,delta2_q50_3,delta2_q50_4,delta2_q50_5,delta2_q50_6,delta2_q50_7,delta2_q50_8,delta2_q50_9,delta2_q50_10,delta2_q50_11,delta2_q50_12,delta2_q75_0,delta2_q75_1,delta2_q75_2,delta2_q75_3,delta2_q75_4,delta2_q75_5,delta2_q75_6,delta2_q75_7,delta2_q75_8,delta2_q75_9,delta2_q75_10,delta2_q75_11,delta2_q75_12,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,109497,-14.466280,99.482414,-13.127948,43.313545,9.995256,5.399690,-5.793285,13.883069,-1.838129,2.187864,-2.674747,3.841628,-9.301169,26.336906,13.115074,12.082673,10.713166,7.633686,5.559512,5.984026,5.907621,5.963255,5.159916,5.725162,4.790164,5.414805,-25.974308,89.797730,-21.092014,38.467827,6.111401,1.586030,-9.500146,9.957172,-5.797652,-1.314428,-6.898641,0.665738,-12.874005,-12.028065,98.771830,-13.556653,44.643456,11.083105,5.215743,-5.622651,14.334005,-1.961504,2.256998,-2.872006,3.641615,-9.346977,0.913519,107.980170,-5.873160,50.600290,15.318432,9.029509,-1.505398,18.285854,1.977354,5.757357,1.137958,7.034313,-5.877173,0.220803,0.009053,-0.002882,-0.003425,0.000750,0.004600,-0.003279,0.011742,0.006375,0.001650,0.000165,-0.000724,0.005746,4.416480,2.527710,1.879202,1.614143,1.372204,0.916860,1.072609,1.153425,1.141011,0.906527,1.142041,0.910949,1.012290,-2.169072,-1.664055,-1.188237,-1.053944,-0.914239,-0.635814,-0.725844,-0.817876,-0.767006,-0.613866,-0.804201,-0.617282,-0.637065,0.089583,0.031641,0.065600,0.032003,0.038310,-0.001774,-0.009710,0.056556,0.040189,-0.014482,-0.020248,0.000350,0.008368,2.173461,1.705773,1.284936,1.063335,0.875187,0.629731,0.694629,0.863577,0.787185,0.644893,0.815830,0.614444,0.648924,-0.135776,-0.035974,0.004075,-0.013851,-0.004475,-0.001072,-0.002500,-0.009171,-0.006136,-0.009151,-0.006148,-0.006779,-0.001375,2.662576,1.463119,1.183564,1.022441,0.859411,0.726815,0.796010,0.860587,0.880833,0.730460,0.8147

In [73]:
# Multi-output Random Forest (built-in support)
rf_multi = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42)

In [74]:
# features
X = merged_df.drop(['track_id'] + continuous_targets, axis=1)

# target groups
target_groups = {
    'rhythm': ['tempo', 'danceability', 'energy'],
    'audio': ['acousticness', 'liveness', 'loudness'],
    'content': ['speechiness', 'instrumentalness', 'valence']
}

# Store results
results = {}

# Train and evaluate each group
for group_name, target_cols in target_groups.items():
    print(f"Training {group_name.upper()} Features")
    
    # targets for this group
    y = merged_df[target_cols]
    
    # Split into train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Train model
    rf_multi = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42)
    rf_multi.fit(X_train, y_train)
    
    # Predict
    predictions = rf_multi.predict(X_test)
    
    # Evaluate each target
    group_results = {}
    for i, col in enumerate(target_cols):
        r2 = r2_score(y_test.iloc[:, i], predictions[:, i])
        mae = mean_absolute_error(y_test.iloc[:, i], predictions[:, i])
        rmse = np.sqrt(mean_squared_error(y_test.iloc[:, i], predictions[:, i]))
        
        group_results[col] = {'r2': r2, 'mae': mae, 'rmse': rmse}
        print(f"{col:20s} R²: {r2:.3f} | MAE: {mae:.3f} | RMSE: {rmse:.3f}")
    
    # Store results and model
    results[group_name] = {
        'model': rf_multi,
        'metrics': group_results,
        'targets': target_cols
    }

# Print summary
print("SUMMARY - Average R² by Group")
for group_name, data in results.items():
    avg_r2 = np.mean([metrics['r2'] for metrics in data['metrics'].values()])
    print(f"{group_name.upper():15s} Average R²: {avg_r2:.3f}")

Training RHYTHM Features


KeyboardInterrupt: 

In [75]:
from sklearn.model_selection import GridSearchCV

# features
X = merged_df.drop(['track_id'] + continuous_targets, axis=1)

# target groups
target_groups = {
    'rhythm': ['tempo', 'danceability', 'energy'],
    'audio': ['acousticness', 'liveness', 'loudness'],
    'content': ['speechiness', 'instrumentalness', 'valence']
}

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# Store results
results = {}

# Train and evaluate each group
for group_name, target_cols in target_groups.items():
    print(f"\n{'='*60}")
    print(f"Training {group_name.upper()} Features with GridSearch")
    print(f"{'='*60}")
    
    # targets for this group
    y = merged_df[target_cols]
    
    # Split into train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # GridSearch with cross-validation
    rf_base = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(
        estimator=rf_base,
        param_grid=param_grid,
        cv=3,  # 3-fold cross-validation
        scoring='r2',
        n_jobs=-1,  # Use all CPU cores
        verbose=1
    )
    
    print(f"Running GridSearch (this may take a few minutes)...")
    grid_search.fit(X_train, y_train)
    
    # Best parameters
    print(f"\nBest parameters for {group_name}:")
    print(grid_search.best_params_)
    print(f"Best CV score: {grid_search.best_score_:.3f}")
    
    # Use best model for predictions
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(X_test)
    
    # Evaluate each target
    print(f"\nTest Set Performance:")
    group_results = {}
    for i, col in enumerate(target_cols):
        r2 = r2_score(y_test.iloc[:, i], predictions[:, i])
        mae = mean_absolute_error(y_test.iloc[:, i], predictions[:, i])
        rmse = np.sqrt(mean_squared_error(y_test.iloc[:, i], predictions[:, i]))
        
        group_results[col] = {'r2': r2, 'mae': mae, 'rmse': rmse}
        print(f"{col:20s} R²: {r2:.3f} | MAE: {mae:.3f} | RMSE: {rmse:.3f}")
    
    # Store results and model
    results[group_name] = {
        'model': best_model,
        'best_params': grid_search.best_params_,
        'cv_score': grid_search.best_score_,
        'metrics': group_results,
        'targets': target_cols
    }



Training RHYTHM Features with GridSearch
Running GridSearch (this may take a few minutes)...
Fitting 3 folds for each of 324 candidates, totalling 972 fits

Best parameters for rhythm:
{'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best CV score: 0.296

Test Set Performance:
tempo                R²: 0.092 | MAE: 21.361 | RMSE: 27.541
danceability         R²: 0.415 | MAE: 0.115 | RMSE: 0.138
energy               R²: 0.535 | MAE: 0.151 | RMSE: 0.181

Training AUDIO Features with GridSearch
Running GridSearch (this may take a few minutes)...
Fitting 3 folds for each of 324 candidates, totalling 972 fits

Best parameters for audio:
{'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}
Best CV score: 0.306

Test Set Performance:
acousticness         R²: 0.462 | MAE: 0.214 | RMSE: 0.253
liveness             R²: 0.039 | MAE: 0.112 | RMSE: 0.148
loudness             R²: 0.5

In [76]:
print("SUMMARY - Average R² by Group")
for group_name, data in results.items():
    avg_r2 = np.mean([metrics['r2'] for metrics in data['metrics'].values()])
    print(f"{group_name.upper():15s} Average R²: {avg_r2:.3f} | CV Score: {data['cv_score']:.3f}")
    print(f"  Best params: {data['best_params']}")

SUMMARY - Average R² by Group
RHYTHM          Average R²: 0.347 | CV Score: 0.296
  Best params: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
AUDIO           Average R²: 0.337 | CV Score: 0.306
  Best params: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}
CONTENT         Average R²: 0.246 | CV Score: 0.201
  Best params: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}


In [ ]:
import joblib
import numpy as np
import os

# Save to parent folder's models directory
models_dir = os.path.join('..', 'models')
os.makedirs(models_dir, exist_ok=True)

# Save your trained models in the parent's models folder
print("Saving models...")
for group_name, data in results.items():
    model_path = os.path.join(models_dir, f'model_{group_name}.pkl')
    joblib.dump(data['model'], model_path)
    print(f"✓ Saved {group_name} model to {model_path}")


# Create a unified predictor class
class UnifiedMusicPredictor:
    def __init__(self, model_paths):
        """Load all group models"""
        self.models = {}
        self.feature_order = ['tempo', 'danceability', 'energy', 
                             'acousticness', 'liveness', 'loudness',
                             'speechiness', 'instrumentalness', 'valence']
        
        for group_name, path in model_paths.items():
            self.models[group_name] = joblib.load(path)
    
    def predict(self, X):
        # Get predictions from each model
        rhythm_pred = self.models['rhythm'].predict(X)  # [tempo, danceability, energy]
        audio_pred = self.models['audio'].predict(X)    # [acousticness, liveness, loudness]
        content_pred = self.models['content'].predict(X) # [speechiness, instrumentalness, valence]
        
        # Combine into single array
        predictions = np.hstack([rhythm_pred, audio_pred, content_pred])
        
        return predictions
    
    def predict_dict(self, X):
        """Return predictions as dictionary for clarity"""
        predictions = self.predict(X)
        
        result = {}
        for i, feature in enumerate(self.feature_order):
            result[feature] = predictions[:, i]
        
        return result

# Initialize the unified predictor with paths to parent's models folder
model_paths = {
    'rhythm': '../models/model_rhythm.pkl',
    'audio': '../models/model_audio.pkl',
    'content': '../models/model_content.pkl'
}

predictor = UnifiedMusicPredictor(model_paths)

# Test it
print("\nTesting Unified Predictor:")
sample_predictions = predictor.predict(X_test)
print(f"Prediction shape: {sample_predictions.shape}")  # Should be (n_samples, 9)
print(f"First sample predictions:\n{sample_predictions[0]}")

# Or get as dictionary
sample_dict = predictor.predict_dict(X_test[:5])
print("\nFirst 5 samples as dictionary:")
for feature, values in sample_dict.items():
    print(f"{feature:20s}: {values[:5]}")

Saving models...
✓ Saved rhythm model to ..\models\model_rhythm.pkl
✓ Saved audio model to ..\models\model_audio.pkl
✓ Saved content model to ..\models\model_content.pkl

Testing Unified Predictor:
Prediction shape: (346, 9)
First sample predictions:
[ 1.18940290e+02  3.61553255e-01  5.16975502e-01  5.47001657e-01
  2.10794069e-01 -1.36840793e+01  6.21974313e-02  5.47757570e-01
  2.12272608e-01]

First 5 samples as dictionary:
tempo               : [118.94029001 129.83866083 113.28319958 120.42865767 126.26880278]
danceability        : [0.36155326 0.42031939 0.44782903 0.37151744 0.4521525 ]
energy              : [0.5169755  0.72352667 0.3895705  0.58921391 0.558611  ]
acousticness        : [0.54700166 0.22107675 0.70529508 0.36972663 0.54253529]
liveness            : [0.21079407 0.2279997  0.16561447 0.2046611  0.18861882]
loudness            : [-13.68407932  -6.31750117 -12.8438341  -10.58172022 -11.22293446]
speechiness         : [0.06219743 0.05797902 0.04918148 0.06400963 0.053766

Cache for faster predictions

In [ ]:
from functools import lru_cache
import hashlib
import pickle

class CachedMusicPredictor(UnifiedMusicPredictor):
    def __init__(self, model_paths, cache_size=1000):
        super().__init__(model_paths)
        self.cache = {}
        self.cache_size = cache_size
    
    def _hash_input(self, X):
        """Create hash of input for cache key"""
        return hashlib.md5(pickle.dumps(X.values if hasattr(X, 'values') else X)).hexdigest()
    
    def predict(self, X, use_cache=True):
        """Predict with optional caching"""
        if not use_cache:
            return super().predict(X)
        
        # Check cache
        cache_key = self._hash_input(X)
        if cache_key in self.cache:
            print("✓ Using cached prediction")
            return self.cache[cache_key]
        
        # Compute prediction
        predictions = super().predict(X)
        
        # Store in cache (with size limit)
        if len(self.cache) >= self.cache_size:
            # Remove oldest entry
            self.cache.pop(next(iter(self.cache)))
        
        self.cache[cache_key] = predictions
        return predictions

# Use cached predictor
cached_predictor = CachedMusicPredictor(model_paths, cache_size=1000)

# First call - computes
predictions1 = cached_predictor.predict(X_test)

# Second call - uses cache
predictions2 = cached_predictor.predict(X_test)  

✓ Using cached prediction
